### Repeat the previous exercise using the extended feature set. Compare the results.

In [89]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
from lxmls import DATA_PATH
import lxmls
import lxmls.sequences.crf_online as crfo
import lxmls.readers.pos_corpus as pcc
import lxmls.sequences.id_feature as idfc
import lxmls.sequences.extended_feature as exfc
from lxmls.readers import pos_corpus

In [91]:
corpus = lxmls.readers.pos_corpus.PostagCorpus()
train_seq = corpus.read_sequence_list_conll(DATA_PATH + "/train-02-21.conll", 
                                            max_sent_len=10, max_nr_sent=1000)

test_seq = corpus.read_sequence_list_conll(DATA_PATH + "/test-23.conll", 
                                           max_sent_len=10, max_nr_sent=1000)

dev_seq = corpus.read_sequence_list_conll(DATA_PATH + "/dev-22.conll", 
                                          max_sent_len=10, max_nr_sent=1000)

In [92]:
feature_mapper = exfc.ExtendedFeatures(train_seq)
feature_mapper.build_features()

In [93]:
for key in feature_mapper.feature_dict:
    if key.startswith('verb_here'):
        print(key)

In [94]:
crf_online = crfo.CRFOnline(corpus.word_dict, corpus.tag_dict, feature_mapper)
crf_online.num_epochs = 20
# crf_online.num_epochs = 3
crf_online.train_supervised(train_seq)


Epoch: 0 Objective value: -7.141596
Epoch: 1 Objective value: -1.807511
Epoch: 2 Objective value: -1.218877
Epoch: 3 Objective value: -0.955739
Epoch: 4 Objective value: -0.807821
Epoch: 5 Objective value: -0.712858
Epoch: 6 Objective value: -0.647382
Epoch: 7 Objective value: -0.599442
Epoch: 8 Objective value: -0.562584
Epoch: 9 Objective value: -0.533411
Epoch: 10 Objective value: -0.509885
Epoch: 11 Objective value: -0.490548
Epoch: 12 Objective value: -0.474318
Epoch: 13 Objective value: -0.460438
Epoch: 14 Objective value: -0.448389
Epoch: 15 Objective value: -0.437800
Epoch: 16 Objective value: -0.428402
Epoch: 17 Objective value: -0.419990
Epoch: 18 Objective value: -0.412406
Epoch: 19 Objective value: -0.405524


In [95]:

pred_train = crf_online.viterbi_decode_corpus(train_seq)
pred_dev = crf_online.viterbi_decode_corpus(dev_seq)
pred_test = crf_online.viterbi_decode_corpus(test_seq)

eval_train = crf_online.evaluate_corpus(train_seq, pred_train, debug=True)
eval_dev = crf_online.evaluate_corpus(dev_seq, pred_dev, debug=True)
eval_test = crf_online.evaluate_corpus(test_seq, pred_test)

And/conj their/pron suspicions/noun of/adp each/det other/adj run/verb deep/adv ./. 
And/conj their/pron suspicions/noun of/adp each/det other/adj run/noun deep/adv ./. 


And/conj there/det 's/verb pride/noun ./. 
And/conj there/det 's/prt pride/noun ./. 


Heiwa/noun Real/noun Estate/noun gained/verb 40/num to/prt 2210/num ./. 
Heiwa/noun Real/adj Estate/noun gained/verb 40/num to/prt 2210/num ./. 


Ms./noun de/noun Vries/noun is/verb a/det free-lance/adj writer/noun ./. 
Ms./noun de/verb Vries/noun is/verb a/det free-lance/adj writer/noun ./. 


Accounting/noun problems/noun raise/verb more/adj knotty/adj issues/noun ./. 
Accounting/noun problems/noun raise/verb more/adv knotty/adj issues/noun ./. 


Pilgrim/noun had/verb been/verb closed/adj for/adp 32/num months/noun ./. 
Pilgrim/noun had/verb been/verb closed/verb for/adp 32/num months/noun ./. 


The/det dividend/noun had/verb been/verb five/num cents/noun a/det share/noun ./. 
The/det dividend/noun had/verb been/verb five/num 

In [96]:

print("CRF - Extended Features Accuracy Train: %.3f Dev: %.3f Test: %.3f" \
      %(eval_train, eval_dev,eval_test))

CRF - Extended Features Accuracy Train: 0.984 Dev: 0.899 Test: 0.894


Compare the errors obtained with the two different feature sets.

- Do some error analysis: what errors were correct by using more features?

- Can you think of other features to use to solve the errors found?